In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

![](./img/common_friends.jpg "")

## Common Friend EdgeList

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row

In [4]:
edgeList = [(1,2), (1,3), (1,4), (2,3), (2,5), (3,4), (3,5), (3,6), (3,7)]

def duplicate_edgeList(edgeList):
    sym_edgeList = []
    for e in edgeList:
        sym_edgeList.append((e[1], e[0]))
    return edgeList + sym_edgeList
    
graphData = sparkSession.sparkContext \
    .parallelize(duplicate_edgeList(edgeList)) \
    .map(lambda(src, dst): Row(src, dst))

![](./img/1.jpg "")

In [5]:
graphSchemaAB = StructType([
    StructField('A', IntegerType(), nullable=False),
    StructField('B', StringType(), nullable=False)
])

ab = sparkSession.createDataFrame(graphData, graphSchemaAB)
ab.show()

+---+---+
|  A|  B|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
|  2|  1|
|  3|  1|
|  4|  1|
|  3|  2|
|  5|  2|
|  4|  3|
|  5|  3|
|  6|  3|
|  7|  3|
+---+---+



In [6]:
graphSchemaBC = StructType([
    StructField('B', IntegerType(), nullable=False),
    StructField('C', StringType(), nullable=False)
])

bc = sparkSession.createDataFrame(graphData, graphSchemaBC)
bc.show()

+---+---+
|  B|  C|
+---+---+
|  1|  2|
|  1|  3|
|  1|  4|
|  2|  3|
|  2|  5|
|  3|  4|
|  3|  5|
|  3|  6|
|  3|  7|
|  2|  1|
|  3|  1|
|  4|  1|
|  3|  2|
|  5|  2|
|  4|  3|
|  5|  3|
|  6|  3|
|  7|  3|
+---+---+



In [7]:
df = ab.join(bc, ab.B == bc.B)
df.show()

+---+---+---+---+
|  A|  B|  B|  C|
+---+---+---+---+
|  3|  7|  7|  3|
|  2|  1|  1|  2|
|  2|  1|  1|  3|
|  2|  1|  1|  4|
|  3|  1|  1|  2|
|  3|  1|  1|  3|
|  3|  1|  1|  4|
|  4|  1|  1|  2|
|  4|  1|  1|  3|
|  4|  1|  1|  4|
|  1|  4|  4|  1|
|  1|  4|  4|  3|
|  3|  4|  4|  1|
|  3|  4|  4|  3|
|  1|  3|  3|  4|
|  1|  3|  3|  5|
|  1|  3|  3|  6|
|  1|  3|  3|  7|
|  1|  3|  3|  1|
|  1|  3|  3|  2|
+---+---+---+---+
only showing top 20 rows



![](./img/4.jpg "")

In [8]:
df = df \
    .drop('B') \
    .groupBy('A', 'C') \
    .count() \
    .filter('A = 1 AND A != C')
df.show()

+---+---+-----+
|  A|  C|count|
+---+---+-----+
|  1|  6|    1|
|  1|  7|    1|
|  1|  3|    2|
|  1|  4|    1|
|  1|  5|    2|
|  1|  2|    1|
+---+---+-----+

